### Import Library
Pertama, import semua library yang dibutuhkan

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from typing import List, Dict

 Menggunakan TfidfVectorizer dari library Scikit-learn.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

Dataset berisi nama, artis, dan lirik untuk dengan jumlah 57650 lagu dalam bahasa Inggris. Data diperoleh dari LyricsFreak.

In [ ]:
songs = pd.read_csv('https://raw.githubusercontent.com/mfaridzia/content-recommendation-system/main/datasets.csv')

Tampilkan data

In [ ]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Karena dataset nya besar, hanya digunakan 500 sample data random saja.

In [ ]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

Membersihkan data yang berisi "\n" 

In [ ]:
songs['text'] = songs['text'].str.replace(r'\n', '')

Menggunakan vektorizer TF-IDF untuk menghitung skor TF-IDF dari setiap lirik lagu, per kata. Dan menentukan argument khusus untuk prediksi.

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

Menggunakan data matrik yang sudah diperoleh untuk prediksi

Untuk menghitung kemiripan lirik, menggunakan cosine similarity

Menghitung kesamaan kosinus dari setiap item dengan setiap item yang lain dalam dataset.

In [ ]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Setelah mendapatkan data kesamaan, maka data nya bisa disimpan dalam dictionary dari 50 lagu paling mirip untuk setiap lagu di dalam data.

In [ ]:
similarities = {}

In [ ]:
for i in range(len(cosine_similarities)):
    # Urutkan setiap elemen dalam cosine_similarities untuk mendapatkan indeks lagu.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # Simpan data yang mirip dari setiap nama dari 50 lagu yang paling mirip. kecuali indeks pertama.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

Gunakan skor kesamaan untuk mengakses item yang paling mirip dan untuk memberikan rekomendasi.

Selanjutnya menentukan kelas untuk pemberi rekomendasi nya.

In [ ]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'{rec_items} lagu yang direkomendasikan untuk {song}:')
        for i in range(rec_items):
            print(f"No {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} dengan skor kesamaan {round(recom_song[i][0], 3)}") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Dapatkan lagu untuk menemukan rekomendasi
        song = recommendation['song']
        # Dapatkan jumlah lagu untuk direkomendasikan
        number_songs = recommendation['number_songs']
        # Dapatkan jumlah lagu yang paling mirip dari kesamaan matriks nya
        recom_song = self.matrix_similar[song][:number_songs]
        self._print_message(song=song, recom_song=recom_song)

Buat instance kelas nya.

In [ ]:
recommedations = ContentBasedRecommender(similarities)

 Pilih lagu dari dataset dan buat rekomendasi.

In [ ]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [ ]:
recommedations.recommend(recommendation)

4 lagu yang direkomendasikan untuk We Here:
No 1:
Thats Not Love by Fabolous dengan skor kesamaan 0.213
--------------------
No 2:
Make U Mine by Fabolous dengan skor kesamaan 0.205
--------------------
No 3:
Do You See by Z-Ro dengan skor kesamaan 0.203
--------------------
No 4:
Work Magic Freestyle by Fabolous dengan skor kesamaan 0.198
--------------------


Pilih lagu random lainnya untuk rekomendasi.

In [ ]:
recommendation2 = {
    "song": songs['song'].iloc[400],
    "number_songs": 4 
}

In [ ]:
recommedations.recommend(recommendation2)

4 lagu yang direkomendasikan untuk On A Slow Boat To China:
No 1:
Boat To Sail by Carpenters dengan skor kesamaan 0.258
--------------------
No 2:
Where The Boat Leaves From by Zac Brown Band dengan skor kesamaan 0.251
--------------------
No 3:
SLOW DOWN by The Jam dengan skor kesamaan 0.209
--------------------
No 4:
Last Chance by Andy Williams dengan skor kesamaan 0.17
--------------------
